<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/beacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 

In [ ]:
base_path = '/content/drive/MyDrive/fixed_train'


In [ ]:
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [ ]:
MAC_Adress = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/'+building+'/*')))
    print(building)
    beacon = list()
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file) as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_BEACON":
                        beacon.append(tmp)
                    
    df = pd.DataFrame(beacon)
    value_counts = df[8].value_counts()
    top_MAC_Adress = value_counts[value_counts > 100].index.tolist()
    print(len(top_MAC_Adress))
    MAC_Adress[building] = top_MAC_Adress
    del df
    del beacon
    gc.collect()

5a0546857ecc773753327266


KeyboardInterrupt: ignored

In [ ]:
ssubm_df

,0,1,2
0,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000000009
1,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000009017
2,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000015326
3,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000018763
4,5a0546857ecc773753327266,046cfa46be49fc10834815c6,0000000022328
...,...,...,...
10128,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000082589
10129,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000085758
10130,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000090895
10131,5dc8cea7659e181adb076a3f,fd64de8c4a2fc5ebb0e9f412,0000000096899


In [ ]:
# Generate the features for the test set

ssubm_building_g = ssubm_df.groupby(0)
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    Px_Power_dfs = list()
    RSSI_dfs = list()
    Distance_dfs = list()
    index = sorted(MAC_Adress[g0.iloc[0,0]])
    feats = list()
    print(gid0)
    for gid,g in g0.groupby(1):

        # get all wifi time locations, 
        with open(os.path.join(base_path, 'test/' + g.iloc[0,1] + '.txt')) as f:
            txt = f.readlines()

        beacon = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_BEACON":
                beacon.append(line)

            
        if not beacon:
            nun_df = pd.DataFrame()
            nun_df.to_csv(gid0+"_beacon_test.csv")
            break
            
        beacon_df = pd.DataFrame(beacon)

        beacon_points = pd.DataFrame(beacon_df.groupby(0).count().index.tolist())
            
        for timepoint in g.iloc[:,2].tolist():

            deltas = (beacon_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            beacon_block_timestamp = beacon_points.iloc[min_delta_idx].values[0]
            
            beacon_block = beacon_df[beacon_df[0] == beacon_block_timestamp].drop_duplicates(subset=8)

            feat_Px_Power = beacon_block.loc[:,[5,8]].set_index(8).reindex(index).replace(np.nan, -999).T.dropna().add_prefix("Tx_Power_")
            feat_RSSI = beacon_block.loc[:,[6,8]].loc[:,[6,8]].set_index(8).reindex(index).replace(np.nan, -999).T.dropna().add_prefix("RSSI_")
            feat_Distance = beacon_block.loc[:,[7,8]].loc[:,[7,8]].set_index(8).reindex(index).replace(np.nan, 0).T.dropna().add_prefix("Distance_")
            
            Px_Power_dfs.append(feat_Px_Power)
            RSSI_dfs.append(feat_RSSI)
            Distance_dfs.append(feat_Distance)

            feat = pd.DataFrame()
            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)

    Px_Power_df = pd.concat(Px_Power_dfs)
    RSSI_df = pd.concat(RSSI_dfs)
    Distance_df = pd.concat(Distance_dfs)
    df = pd.concat(feats)
    building_df = pd.concat([Px_Power_df.reset_index(), RSSI_df.reset_index(), Distance_df.reset_index(), df.reset_index()], axis=1)    
    building_df.to_csv(gid0+"_beacon_test.csv")

    

In [ ]:
nun_df

NameError: ignored